In [1]:
import glob
import japanize_matplotlib
import os
import pandas as pd
import random
import sys
import math
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as sp
from sklearn.metrics import r2_score

In [2]:
def return_fixed_class(BenchMark="bt", Processes=[1, 2, 4, 8, 16, 32, 64, 128, 256], FixedBenchMarkClass="C"):
        path = './'
        
        # fixed_Class にはベンチマーククラスFixedBenchMarkClassで実行プロセス数がProcessesに該当するものの結果が入る
        fixed_Class = list()

        for process in Processes:
            file_name = ("pprof_"+BenchMark+FixedBenchMarkClass+str(process)+".csv")
            file_path = path+file_name
            if (os.path.exists(file_path) and os.stat(file_path).st_size != 0):
                data_frame = pd.read_csv(path+file_name)
                data_frame = data_frame.set_index(['Name'])
                fixed_Class.append(data_frame.rename(columns = {'#Call': process}).sort_index())
        return(fixed_Class)
    

In [3]:
def return_fixed_process(BenchMark="bt", BenchMarkClasses=["S", "W", "A", "B", "C", "D"], FixedProcess=32):
    path = './'

    # fixed_process には実行プロセス数が64でベンチマーククラスがA ~ Dまでの結果が入る
    fixed_process = list()

    for bench_mark_class in BenchMarkClasses:
        file_name = ("pprof_"+BenchMark+bench_mark_class+str(FixedProcess)+".csv")
        file_path = path+file_name
        if (os.path.exists(file_path) and os.stat(file_path).st_size != 0):
            data_frame = pd.read_csv(path+file_name)
            data_frame = data_frame.set_index(['Name'])
            fixed_process.append(data_frame.rename(columns = {'#Call': bench_mark_class}).sort_index())
    return(fixed_process)
       

# ベンチマーク名：CGを対象にDataFrameとして集計したsummary_fixed_classの重複データを統合する実験

In [4]:
BenchMark_name = "cg"
cg_fixed_class = return_fixed_class(BenchMark=BenchMark_name)
cg_fixed_process = return_fixed_process(BenchMark=BenchMark_name)
summary_fixed_class = pd.concat(cg_fixed_class, axis=1)

In [29]:
dropped_summary_fixed_class = summary_fixed_class.drop_duplicates()
print(dropped_summary_fixed_class)

print(summary_fixed_class)

                        1          2          4          8          16   \
Name                                                                      
.TAU_application        1.0        1.0        1.0        1.0        1.0   
CONJ_GRAD              76.0       76.0       76.0       76.0       76.0   
ICNVRT            1984770.0  2263540.0  2821070.0  3936140.0  3936140.0   
MPI_Irecv()         12236.0    12483.0    10982.0    13984.0    13984.0   
SPRNVC              84375.0    93750.0   112500.0   150000.0   150000.0   

                        32         64         128        256  
Name                                                          
.TAU_application        1.0        1.0        1.0        1.0  
CONJ_GRAD              76.0       76.0       76.0       76.0  
ICNVRT            3936140.0  3936140.0  3936140.0  3936140.0  
MPI_Irecv()         19988.0    19988.0    25992.0    25992.0  
SPRNVC             150000.0   150000.0   150000.0   150000.0  
                            1    

In [27]:
def evaluate_dataframes(df1, df2):
    df1_index = df1.index.values[0]
    df2_index = df2.index.values[0]
#     print("-----print(df1.values.tolist())-----\n")
#     print(df1.values.tolist())
#     print("-----print(df1.values.tolist()[0][1])")
#     print(df1.values.tolist()[0][1])
#     print("-----print(len(df1.values.tolist()[0]))")
#     print(len(df1.values.tolist()[0]))
    
    for i in range(len(df1.values.tolist()[0])):
        if(df1.values.tolist()[0][i] != df2.values.tolist()[0][i]):
            return False
    return True

In [28]:
for dropped_index_name in summary_fixed_class.drop_duplicates().index.values:
    dropped_index_name_data = summary_fixed_class.loc[[dropped_index_name]]
    replace_name = dropped_index_name
    for all_index_name in summary_fixed_class.index.values:
        all_index_name_data = summary_fixed_class.loc[[all_index_name]]
        if(dropped_index_name == all_index_name):
            pass
        elif(evaluate_dataframes(dropped_index_name_data, all_index_name_data)):
            replace_name += f", {all_index_name}"
    print(f"replace_name ===== {replace_name}\n")

replace_name ===== .TAU_application, ALLOC_SPACE, CG, INITIALIZE_MPI, MAKEA, MPI_Barrier(), MPI_Bcast(), MPI_Comm_rank(), MPI_Comm_size(), MPI_Finalize(), MPI_Init(), MPI_Reduce(), SETUP_PROC_INFO, SETUP_SUBMATRIX_INFO, SPARSE

replace_name ===== CONJ_GRAD

replace_name ===== ICNVRT

replace_name ===== MPI_Irecv(), MPI_Send(), MPI_Wait()

replace_name ===== SPRNVC, VECSET



In [ ]:
print(dropped_index_name_data)
print(all_index_name_data)

In [ ]:
print(type(dropped_index_name_data))
print(type(all_index_name_data))
if(dropped_index_name_data.values.all() == all_index_name_data.values.all()):
    print("equal")

In [ ]:
columns = summary_fixed_class.columns.to_numpy()

index = summary_fixed_class.index.to_numpy()

dict_summary_fixed_class = {"benchmarks": columns}

for index_name in index:
    dict_summary_fixed_class[index_name] = summary_fixed_class.T[index_name].to_numpy()